# Probabilities and k-Means Clustering

Using the IMDB data, construct a feature matrix, and apply `k-Means` to the data to extract clusters. 

We then inspect various aspects of probability associated with these clusterings.

In [1]:
import json

import pandas as pd
import numpy as np

In [3]:
actor_name_map = {}
movie_actor_map = {}
actor_genre_map = {}


with open("imdb_movies_2000to2022.prolific.json", "r") as in_file:
    for line in in_file:
        
        # Read the movie on this line and parse its json
        this_movie = json.loads(line)
                    
        # Add all actors to the id->name map
        for actor_id,actor_name in this_movie['actors']:
            actor_name_map[actor_id] = actor_name
            
        # For each actor, add this movie's genres to that actor's list
        for actor_id,actor_name in this_movie['actors']:
            this_actors_genres = actor_genre_map.get(actor_id, {})
            
            # Increment the count of genres for this actor
            for g in this_movie["genres"]:
                this_actors_genres[g] = this_actors_genres.get(g, 0) + 1
                
            # Update the map
            actor_genre_map[actor_id] = this_actors_genres
            
        # Finished with this film
        movie_actor_map[this_movie["imdb_id"]] = ({
            "movie": this_movie["title"],
            "actors": set([item[0] for item in this_movie['actors']]),
            "genres": this_movie["genres"]
        })

In [4]:
print("Known Actors:", len(actor_name_map))
print("Known Movies:", len(movie_actor_map))

Known Actors: 33609
Known Movies: 20620


## Read CSV of Movies to Cluster IDs

Using the provided movie-to-cluster mapping CSV file, we assess the distributions of movies per cluster and ask questions about genres and actors in each cluster.

In [5]:
cluster_df = pd.read_csv("movie_to_cluster.csv")

In [6]:
cluster_df["cluster"].value_counts()

6     3177
0     3097
15    1754
13    1705
2     1503
12    1466
1     1376
3     1240
14     893
8      774
10     761
4      655
11     640
7      635
5      560
9      384
Name: cluster, dtype: int64

In [7]:
cluster_pr_map = {cluster_id:cluster_pr for cluster_id,cluster_pr in (cluster_df["cluster"].value_counts() / cluster_df.shape[0]).items()}
cluster_pr_df = pd.DataFrame(cluster_df["cluster"].value_counts() / cluster_df.shape[0])

cluster_pr_df

,cluster
6,0.154074
0,0.150194
15,0.085063
13,0.082687
2,0.072890
12,0.071096
1,0.066731
3,0.060136
14,0.043307
8,0.037536


## Assess Genre-Specific Cluster Probabilities

We want to determine, for a new movie with a known genre, to which cluster is it most likely to be assigned?

In [8]:
# For each genre, count the number of movies
genre_counts = {}

# For each movie, get its genres and update the genre count
for movie_id in movie_actor_map.keys():
    for genre in movie_actor_map[movie_id]["genres"]:
        genre_counts[genre] = genre_counts.get(genre, 0) + 1
        
genre_prs = []
for genre,g_count in genre_counts.items():
    genre_prs.append((genre, g_count/len(movie_actor_map)))
    
genre_prs_df = pd.DataFrame(genre_prs, columns=["genre", "probability"])
genre_pr_map = {row["genre"]:row["probability"] for idx,row in genre_prs_df.iterrows()}

genre_prs_df.sort_values(by="probability", ascending=False)

,genre,probability
5,Drama,0.492580
0,Comedy,0.291804
10,Thriller,0.194180
6,Action,0.181523
3,Horror,0.149224
8,Crime,0.134481
2,Romance,0.122260
7,Adventure,0.080844
9,Mystery,0.074442
4,Sci-Fi,0.051164


In [9]:
#Setting the genre we will be investigating
# target_genre = "Sci-Fi"
# target_genre = "Western"
target_genre = "Musical"

### Calculate Conditional Probabilities of the given Genre for each cluster

We want to calculate the conditional probability of a movie being assigned to a cluster given its genre. This probability can be calculated as follows:

$$ Pr(Y=Cluster <ID> | X=Genre) = \frac{Pr(X=Genre | Y=Cluster <ID>) Pr(Y=Cluster <ID>)}{Pr(X=Genre)}  $$

We have $Pr(X=Genre)$ above via `genre_prs_df`, and we have $Pr(Y=Cluster <ID>)$ above via `cluster_pr_df`, so all we need to calculate $Pr(X=Genre | Y=Cluster <ID>)$.


In [11]:
per_cluster_prs = []
for cluster_id,group in cluster_df.groupby("cluster"):

    this_cluster_genre_count = sum([
        1 if target_genre in movie_actor_map[m]["genres"] else 0 
        for m in group["movie_id"]
    ])
    
    # Calculate conditional probability of the target genre given this cluster
    ## Pr(X=Genre | Y=Cluster <ID>)
    pr_genre_given_cluster = this_cluster_genre_count / group.shape[0]
    print("Pr[%s| Cluster %02d]:" % (target_genre, cluster_id), "\t", pr_genre_given_cluster)
    
    # Then add this calculated probability to `per_cluster_prs`,
    ## so we can track this conditional probability
    joint_pr_genre_cluster = pr_genre_given_cluster * group.shape[0] / cluster_df.shape[0]
    print("Pr[%s, Cluster %02d]:" % (target_genre, cluster_id), "\t", joint_pr_genre_cluster)
    per_cluster_prs.append(joint_pr_genre_cluster)

Pr[Musical| Cluster 00]: 	 0.018727801097836616
Pr[Musical, Cluster 00]: 	 0.002812803103782735
Pr[Musical| Cluster 01]: 	 0.002180232558139535
Pr[Musical, Cluster 01]: 	 0.0001454898157129001
Pr[Musical| Cluster 02]: 	 0.001996007984031936
Pr[Musical, Cluster 02]: 	 0.0001454898157129001
Pr[Musical| Cluster 03]: 	 0.0064516129032258064
Pr[Musical, Cluster 03]: 	 0.0003879728419010669
Pr[Musical| Cluster 04]: 	 0.0
Pr[Musical, Cluster 04]: 	 0.0
Pr[Musical| Cluster 05]: 	 0.0035714285714285713
Pr[Musical, Cluster 05]: 	 9.699321047526673e-05
Pr[Musical| Cluster 06]: 	 0.009757632987094743
Pr[Musical, Cluster 06]: 	 0.0015033947623666344
Pr[Musical| Cluster 07]: 	 0.0
Pr[Musical, Cluster 07]: 	 0.0
Pr[Musical| Cluster 08]: 	 0.016795865633074936
Pr[Musical, Cluster 08]: 	 0.0006304558680892338
Pr[Musical| Cluster 09]: 	 0.0
Pr[Musical, Cluster 09]: 	 0.0
Pr[Musical| Cluster 10]: 	 0.002628120893561104
Pr[Musical, Cluster 10]: 	 9.699321047526673e-05
Pr[Musical| Cluster 11]: 	 0.0
Pr[Mus

In [12]:
# For each cluster ID, calculate the posterior probability given the target genre
for cluster_id,cluster_genre_pr in enumerate(per_cluster_prs):

    # combine this cluster_genre_pr, cluster_pr_df, and genre_prs_df
    ## to calculate Pr(Cluster|Genre)
    pr_cluster_given_genre = cluster_genre_pr / genre_pr_map[target_genre]

    print("Pr[Cluster %02d | %s]:" % (cluster_id, target_genre), "\t", pr_cluster_given_genre)
    

Pr[Cluster 00 | Musical]: 	 0.3452380952380952
Pr[Cluster 01 | Musical]: 	 0.017857142857142856
Pr[Cluster 02 | Musical]: 	 0.017857142857142856
Pr[Cluster 03 | Musical]: 	 0.047619047619047616
Pr[Cluster 04 | Musical]: 	 0.0
Pr[Cluster 05 | Musical]: 	 0.011904761904761904
Pr[Cluster 06 | Musical]: 	 0.1845238095238095
Pr[Cluster 07 | Musical]: 	 0.0
Pr[Cluster 08 | Musical]: 	 0.07738095238095238
Pr[Cluster 09 | Musical]: 	 0.0
Pr[Cluster 10 | Musical]: 	 0.011904761904761904
Pr[Cluster 11 | Musical]: 	 0.0
Pr[Cluster 12 | Musical]: 	 0.03571428571428571
Pr[Cluster 13 | Musical]: 	 0.11309523809523808
Pr[Cluster 14 | Musical]: 	 0.1369047619047619
Pr[Cluster 15 | Musical]: 	 0.0


### Sample Titles in Each Cluster

We can use the above conditional probabilities to determine the most likely cluster given a movie genre.

Here, we sample movies in the most likely cluster to get a sense of what movies are in that cluster.

In [ ]:
target_cluster = 6

In [ ]:
for movie_id in cluster_df[cluster_df["cluster"] == target_cluster].sample(n=10, replace=False)["movie_id"]:
    this_movie = movie_actor_map[movie_id]
    print(movie_id, this_movie["movie"], this_movie["genres"])

## Assess Actor-Specific Cluster Probabilities

Above, we determine the most likely cluster given a movie genre. Here, we ask the same question for a given actor.

In [ ]:
# For each actor, count the number of movies
actor_counts = {}

# For each movie, get its genres and update the genre count
for movie_id in movie_actor_map.keys():
    for actor in movie_actor_map[movie_id]["actors"]:
        actor_counts[actor] = actor_counts.get(actor, 0) + 1
        
actor_prs = []
for actor,a_count in actor_counts.items():
    actor_prs.append((actor, a_count/len(movie_actor_map)))
    
actor_prs_df = pd.DataFrame(actor_prs, columns=["actor", "probability"])
actor_pr_map = {row["actor"]:row["probability"] for idx,row in actor_prs_df.iterrows()}

actor_prs_df.sort_values(by="probability", ascending=False)

In [ ]:
#Setting the actor we will be comparing to
# target_actor_id = 'nm1165110' # Chris Hemsworth
# target_actor_id = 'nm0413168' # Hugh Jackman
# target_actor_id = 'nm0005351' # Ryan Reynolds
# target_actor_id = "nm0000206" # Keanu Reeves
target_actor_id = 'nm0000115' # Nic Cage

In [ ]:
per_cluster_prs = []
for cluster_id,group in cluster_df.groupby("cluster"):

    this_cluster_actor_count = sum([
        1 if target_actor_id in movie_actor_map[m]["actors"] else 0 
        for m in group["movie_id"]
    ])
    
    # Calculate conditional probability of the target actor given this cluster
    ## Pr(X=Actor | Y=Cluster <ID>)
    raise("INSERT CODE HERE")
    
    # Then add this calculated probability to `per_cluster_prs`,
    ## so we can track this conditional probability
    per_cluster_prs.append(raise("INSERT CODE HERE"))

In [ ]:
# For each cluster ID, calculate the posterior probability given the target actor
for cluster_id,cluster_actor_pr in enumerate(per_cluster_prs):

    # combine this cluster_actor_pr, cluster_pr_df, and actor_prs_df
    ## to calculate Pr(Cluster|Actor)
    pr_cluster_given_actor = raise("INSERT CODE HERE")

    print("Pr[Cluster %02d | %s]:" % (cluster_id, target_actor), "\t", pr_cluster_given_actor)
    